<a href="https://colab.research.google.com/github/achiral/YouTubeCapE2J/blob/main/YouTubeCapE2J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声・動画ファイルから文字起こし、翻訳、字幕挿入
@sakasegawa（https://qiita.com/sakasegawa）様、Nego_t（https://zenn.dev/nego_t/articles/7c2c11f9bed467）様の記事を参考に、少し編集を加えました。  
**処理の内容自体は、まったく同じ**です。  
以下の操作をできるように整理したものです。
- Google Driveに保存してある動画ファイル、音声ファイルを利用
- 音声ファイルの処理
  - 文字起こし
  - 翻訳（英語から日本語）
  - テキストファイルへの書き出し
- 動画ファイルの処理
  - 対応する音声ファイルが必要
    - FFmpegでできそう
    - 自作動画であればYouTubeにアップロードして一括処理可能
  - 字幕挿入
    - 動画に対応した音声ファイルが必要
    - 動画から音声分離の処理が必要 

手持ち動画に字幕挿入するには、もう少し修正の必要がありそうです。気が向いたらFFmpegで音声分離するコードを追記してみます。

## オリジナルサイト
- 概要  
YouTube英語動画に自動で字幕をつける(動画ライセンスに注意)
https://qiita.com/sakasegawa/items/1eae5038cfbd09baad44

- 使い方  
https://qiita.com/sakasegawa/items/1eae5038cfbd09baad44

- サンプル  
https://www.youtube.com/watch?v=R-qLOUTHP5Q

In [1]:
#@title パスの設定
# YouTubeのvidを入力する(入力しない場合はデフォルト)
url = "https://www.youtube.com/watch?v=WRxN3omvrf0"#@param {type:"string"}{allow-input: true} ["YouTubeのURLを入力してください","https://www.youtube.com/watch?v=WRxN3omvrf0", "https://www.youtube.com/watch?v=[INPUT]"]
# 動画はPlayground/videos以下に保存される
video_folder = "/content/drive/MyDrive/movietranslate/videos/"#@param {type:"string"}{allow-input: true}["インプットファイルのフォルダパスを入力してください", "/content/drive/MyDrive/movietranslate/videos/"]
# フォントは以下のフォントをPlayground/fonts以下に置く
# https://flopdesign.booth.pm/items/2328262
# LightNovelPOPv2.otf として保存
font_folder = "/content/drive/MyDrive/movietranslate/fonts/"#@param {type:"string"}{allow-input: true}["アウトプット先のフォルダパスを入力してください", "/content/drive/MyDrive/movietranslate/fonts/"]
if url == "":
  url = "https://www.youtube.com/watch?v=WRxN3omvrf0"

print("video_folder = " + video_folder)
print("font_folder = " + font_folder)
print("url = " + url)

video_folder = /content/drive/MyDrive/movietranslate/videos/
font_folder = /content/drive/MyDrive/movietranslate/fonts/
url = https://www.youtube.com/watch?v=WRxN3omvrf0


In [2]:
#@title url to vid
import urllib.parse
qs = urllib.parse.urlparse(url).query
qs_d = urllib.parse.parse_qs(qs)
vid = 'WRxN3omvrf0' #@param ["qs_d['v'][0]", "拡張子を除いたファイル名を入力してください(例)WRxN3omvrf0", "'WRxN3omvrf0'"] {type:"raw", allow-input: true}
print("url: https://youtube.com/watch?v=%s" % vid)
print("vid: " + vid)

url: https://youtube.com/watch?v=WRxN3omvrf0
vid: WRxN3omvrf0


In [3]:
#@title nvidia確認
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e351d85f-93da-e6d0-c61e-dd77a70ddfce)


In [4]:
#@title Mount Google Drive
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

import os
if not os.path.exists("/content/drive/MyDrive/movietranslate/fonts/LightNovelPOPv2.otf"):
    raise Exception("フォルダを作成してフォントをダウンロードしてください")
print("○フォントフォルダーの内容：")
%ls /content/drive/MyDrive/movietranslate/fonts

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
○フォントフォルダーの内容：
LanobePOPv2.zip  LightNovelPOPv2.otf


In [5]:
#@title インストール
# youtube: 動画ダウンロード
!pip install yt_dlp
# 音声処理系: whisper
!pip install pydub
!pip install git+https://github.com/openai/whisper.git
# 動画処理系: moviepy
!pip install -q moviepy
!apt install imagemagick
!pip install imageio==2.4.1
# その他: いい感じの改行
!pip install budoux
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cq72nrdj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-cq72nrdj
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  

In [6]:
#@title おまじない
# https://github.com/Zulko/moviepy/issues/693#issuecomment-622997875
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

In [ ]:
#@title 音声処理: ダウンロード (ダウンロード済みファイルがあれば処理不要)
import yt_dlp
from pydub import AudioSegment

# 動画全体の音声をダウンロード
# videos/{vid}.m4a として出力している
def music_download(vid):
    output_file_name = video_folder + f"{vid}.m4a"
    video_url = f"https://www.youtube.com/watch?v={vid}"

    ydl_opts = {
        "format": "m4a/bestaudio/best",
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "m4a",
            }
        ],
        "outtmpl": output_file_name,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(video_url)
    
    audio = AudioSegment.from_file(output_file_name, "m4a")
    audio_file_name = video_folder + f"{vid}.wav"
    audio.export(audio_file_name, format="wav")

In [ ]:
#@title 動画処理: ダウンロード (ダウンロード済みファイルがあれば処理不要)

import yt_dlp
from yt_dlp.utils import download_range_func

# videos/{vid}.mp4 として出力している
def video_download(vid):
    output_file_name = video_folder + f"{vid}.mp4"
    video_url = "https://www.youtube.com/watch?v=%s" % vid

    ydl_opts = {
        "format": "bestvideo[ext=mp4]+bestaudio[ext=mp4]/mp4+best[height<=480]",
        "outtmpl": output_file_name
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(video_url)

In [22]:
#@title whisper
# 音声処理: whisper
import whisper

def generate_transcribe_wav(vid):
    # Whisper高速化テクニック
    # https://qiita.com/halhorn/items/d2672eee452ba5eb6241
    model = whisper.load_model("large", device="cpu")
    _ = model.half()
    _ = model.cuda()

    # exception without following code
    # reason : model.py -> line 31 -> super().forward(x.float()).type(x.dtype)
    for m in model.modules():
        if isinstance(m, whisper.model.LayerNorm):
            m.float()
    target_file_name = video_folder + f"{vid}.wav"
    result = model.transcribe(target_file_name)
    return result

def generate_transcribe_mp3(vid):
    # Whisper高速化テクニック
    # https://qiita.com/halhorn/items/d2672eee452ba5eb6241
    model = whisper.load_model("large", device="cpu")
    _ = model.half()
    _ = model.cuda()

    # exception without following code
    # reason : model.py -> line 31 -> super().forward(x.float()).type(x.dtype)
    for m in model.modules():
        if isinstance(m, whisper.model.LayerNorm):
            m.float()
    target_file_name = video_folder + f"{vid}.mp3"
    result = model.transcribe(target_file_name)
    return result

In [8]:
#@title 翻訳モデル: fugumt
from transformers import pipeline

translator = pipeline('translation', model='staka/fugumt-en-ja')

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
#@title 動画処理: 字幕(日本語の場合は seg_text: 「seg_text」 を選択) {display-mode: "form"}
from moviepy import editor
from moviepy.video.io.VideoFileClip import VideoFileClip
import budoux

def generate_subtitle(vid, transcribe, max_line_length = 14, max_lines = 2):
    input_file_name = '/content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0.mp4'             #@param ["video_folder + f'{vid}.mp4'", "パスを入力するか選択してください: /content/drive/MyDrive/movietranslate/videos/FILENAME.mp4", "'/content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0.mp4'"] {type:"raw", allow-input: true}
    output_file_name = '/content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0_subtitle2.mp4' #@param ["video_folder + f'{vid}_subtitle.mp4'", "パスを入力するか選択してください: /content/drive/MyDrive/movietranslate/videos/FILENAME_subtitle.mp4", "'/content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0_subtitle2.mp4'"] {type:"raw", allow-input: true}
    video = VideoFileClip(input_file_name)
    annotated_clips = []
    last_end = 0
    for segment in transcribe['segments']:
        seg_start_sec = int(segment['start'])
        seg_end_sec = int(segment['end'])
        seg_text = segment['text']
        # 翻訳
        result = translator(seg_text)
        seg_text = result[0]['translation_text'] #@param ["result[0]['translation_text']", "seg_text"] {type:"raw", allow-input: true}
        # 字幕がない場所はそのまま動画を入れる
        if last_end != seg_start_sec:
            print("skip: %s to %s" % (last_end, seg_start_sec))
            annotate_seg = video.subclip(last_end, seg_start_sec)
            annotated_clips.append(annotate_seg)

        # 文章が長すぎる場合は分割しつつ字幕として表示する
        seg_all_sec = seg_end_sec - seg_start_sec
        parsed_text_list = sentence_parse_and_line_parse(seg_text, max_line_length, max_lines)
        section_per_sec = int(seg_all_sec / len(parsed_text_list))
        print("jimaku: %s to %s" % (seg_start_sec, seg_end_sec))
        if section_per_sec == 0:
            section_per_sec = 1
        for i, parsed_text in enumerate(parsed_text_list):
            section_start_sec = seg_start_sec + section_per_sec * i
            section_end_sec = seg_start_sec + section_per_sec * (i + 1)
            # 一番ケツはend_secに合わせる
            if i == (len(parsed_text_list) - 1):
                section_end_sec = seg_end_sec
                print("    ketsu: %s" % section_end_sec)
            # 最後ちょびっとでも超えるとエラーなので一番最後はNoneにする
            if section_end_sec >= video.duration:
                section_end_sec = None
            annotate_seg = annotate(video.subclip(section_start_sec, section_end_sec), parsed_text)
            print("    section start: %s, end: %s, text: %s" % (section_start_sec, section_end_sec, parsed_text))
            annotated_clips.append(annotate_seg)

        # last_end: 前回の終わり秒の参照用
        last_end = seg_end_sec
        print(last_end)
    
    print(f"last_end is {last_end} and video.duration is {video.duration}")
    if last_end != None and video.duration >= last_end:
        print("skip: %s to %s" % (last_end, video.duration))
        annotate_seg = video.subclip(last_end, None)
        annotated_clips.append(annotate_seg)
    final_clip = editor.concatenate_videoclips(annotated_clips)
    final_clip.write_videofile(output_file_name, audio_codec="aac")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2400256/45929032 bytes (5.2%)5472256/45929032 bytes (11.9%)8454144/45929032 bytes (18.4%)11206656/45929032 bytes (24.4%)14589952/45929032 bytes (31.8%)17203200/45929032 bytes (37.5%)20299776/45929032 bytes (44.2%)23552000/45929032 bytes (51.3%)26836992/45929032 bytes (58.4%)30294016/45929032 bytes (66.0%)32595968/45929032 bytes (71.0%)35741696/45929032 bytes (77.8%)

In [10]:
#@title 字幕の設定関数 {display-mode: "form" }
def annotate(clip, txt):
    # https://moviepy-tburrows13.readthedocs.io/en/improve-docs/ref/VideoClip/TextClip.html
    # 1外側
    txtclip1 = editor.TextClip(txt
        ,fontsize=75                                #@param {type:"integer", allow-input: true}
        ,font=font_folder + 'LightNovelPOPv2.otf'   #@param ["font_folder + 'LightNovelPOPv2.otf'", ""]{type:"raw", allow-input: true}
        ,color='white'                            #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        ,stroke_color="white"                  #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        ,stroke_width=12                           #@param {type:"integer", allow-input: true}
    )
    # 2内側
    txtclip2 = editor.TextClip(txt
        , fontsize=75                               #@param {type:"integer", allow-input: true}
        , font=font_folder + 'LightNovelPOPv2.otf'  #@param ["font_folder + 'LightNovelPOPv2.otf'", ""]{type:"raw", allow-input: true}
        , color='yellow'                              #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        , stroke_color="red"                      #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        , stroke_width=5                          #@param {type:"integer", allow-input: true}
    )
    # 3
    # txtclip3 = editor.TextClip(txt
    #     ,  fontsize=75                               #@param {type:"integer", allow-input: true}
    #     ,  font=font_folder + 'LightNovelPOPv2.otf'  #@param ["font_folder + 'LightNovelPOPv2.otf'", ""]{type:"raw", allow-input: true}
    #     ,  color='white'                              #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
    #     ,  stroke_color="white"                      #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
    #     ,  stroke_width=12                          #@param {type:"integer", allow-input: true}
    # )
    cvc = editor.CompositeVideoClip([clip
                                     ,txtclip1.set_pos(('center', 'bottom'))
                                     ,txtclip2.set_pos(('center', 'bottom'))
                                    #  ,txtclip3.set_pos(('center', 'bottom'))
                                     ])
    return cvc.set_duration(clip.duration)

# 文章をいい感じに分割・改行する
# 例
# input: "いいね あのあくまでそのすでにその下預金で2600万円とか置いてる人であれば多分500万円ぐらい"
# output: ['いいね', 'あのあくまでそのすでにその\n下預金で2600万円とか', '置いてる人であれば\n多分500万円ぐらい']
def sentence_parse_and_line_parse(text, max_line_length = 14, max_lines = 2):
    # スペースで分割
    space_parsed_text_list = text.split(' ')
    parsed_text_list = []
    for space_parsed_text in space_parsed_text_list:
        parsed_text_list += text_line_parse(space_parsed_text, max_line_length, max_lines)
    return parsed_text_list

# 任意の行数ごとにわける
# 例
# input: "あのあくまでそのすでにその下預金で2600万円とか置いてる人であれば多分500万円ぐらい"
# output: ['あのあくまでそのすでにその\n下預金で2600万円とか', '置いてる人であれば\n多分500万円ぐらい']
def text_line_parse(text, max_length = 14, max_lines = 2):
    text_list = budoux_parse_text(text, max_length, [])
    line_parsed_text_list = []
    for i in range(0, len(text_list), max_lines):
        text_with_2lines = text_list[i: i+max_lines]
        text = "\n".join(text_with_2lines)
        line_parsed_text_list.append(text)
    return line_parsed_text_list

# budouxで任意の文字数ごとにわける
# 最後の引数は再帰用
# 例
# input: "あのあくまでそのすでにその下預金で2600万円とか置いてる人であれば多分500万円ぐらい"
# output: ['あのあくまでそのすでにその', '下預金で2600万円とか', '置いてる人であれば', '多分500万円ぐらい']
def budoux_parse_text(text, max_length = 14, text_list = []):
    budoux_parser = budoux.load_default_japanese_parser()
    budoux_parsed_text_list = budoux_parser.parse(text)
    parsed_text = ""
    overflow_text = ""
    for budoux_parsed_text in budoux_parsed_text_list:
        if max_length > len(parsed_text + budoux_parsed_text):
            parsed_text += budoux_parsed_text
        elif len(parsed_text) == 0 and len(budoux_parsed_text) > max_length:
            parsed_text = budoux_parsed_text[0:max_length]
            overflow_text += budoux_parsed_text[max_length:]
        else:
            overflow_text += budoux_parsed_text
    text_list.append(parsed_text)
    if len(overflow_text) > 0:
        print(overflow_text)
        budoux_parse_text(overflow_text, max_length, text_list)
    return text_list

In [21]:
#@title 動画から音声（mp3）抽出
input_path = video_folder + vid + '.mp4' #@param ["video_folder + vid + '.mp4'"] {type:"raw", allow-input: true}
out_path = video_folder + vid + '.mp3' #@param ["video_folder + vid + '.mp3'", "video_folder + vid + '.wav'", "video_folder + vid + '.aac'"] {type:"raw", allow-input: true}

# !ffmpeg -i $input_path -vn -acodec copy $out_path
# !ffmpeg -i $input_path -vn -acodec copy $out_path
!ffmpeg -i $input_path -acodec libmp3lame -ab 256k $out_path

print("「" + input_path + "」の動画ファイルから")
print("「" + out_path + "」に音声抽出しました")

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
#@title 音声・動画ダウンロード (ダウンロード済みファイルがあれば処理不要: コメントアウト), 音声書き出し
# URLの確認
print("◯対象URL")
print("https://youtube.com/watch?v=%s" % vid)
# 音声をダウンロードする
print("◯音声ダウンロード")
print("...")
# music_download(vid)
print("音声ダウンロード完了!")
# 音声書き出し
print("◯音声書き出し")
print("...")
transcribe = generate_transcribe_wav(vid)       #@param ["generate_transcribe(vid)", "generate_transcribe_mp3(vid)"] {type:"raw", allow-input: true}
print("音声書き出し完了！")
# 動画をダウンロードする
print("◯動画ダウンロード")
# video_download(vid)

◯対象URL
https://youtube.com/watch?v=WRxN3omvrf0
◯音声ダウンロード
...
音声ダウンロード完了!
◯音声書き出し
...


In [ ]:
#@title save original text
print("○google driveに書き出します○")
output_file = "result.txt"
txt_path = video_folder + output_file
with open(txt_path, mode='w') as f:
  f.write(transcribe["text"])

print("●「" + txt_path + "」 に以下のテキストを出力しました●")
print(transcribe["text"])

In [ ]:
#@title translation (E2J) -> save translated text
# translation (E2J)
print("○翻訳中です(English to Japanese)")
result_translated = translator(transcribe['text'])
print("●翻訳が終了しました●")
# save
print("○google driveに書き出します○")
output_file = "result_translated.txt"
txt_path = video_folder + output_file
with open(txt_path, mode='w') as f:
  f.write(result_translated[0]["translation_text"])

print("●「" + txt_path + "」 に以下のテキストを出力しました●")
print(result_translated[0]["translation_text"])

In [14]:
#@title 動画に字幕データをつけて出力する
print("◯字幕生成")
generate_subtitle(vid, transcribe)

◯字幕生成
ジョシュです。タコマ・コミュニティ・カレッジの学生です。
タコマ・コミュニティ・カレッジの
ジの
jimaku: 0 to 3
    section start: 0, end: 1, text: こんにちは、私は私は
ジョシュです。学生です。
    ketsu: 3
    section start: 1, end: 3, text: タコマ・コミュニティ・カレッ
ジの
3
skip: 3 to 5
コロラド州でスノーボードのインストラクターをしていました。
インストラクターをしていました。
していました。
jimaku: 5 to 9
    section start: 5, end: 7, text: 学校に戻る前は、
コロラド州でスノーボードの
    ketsu: 9
    section start: 7, end: 9, text: インストラクターを
していました。
9
skip: 9 to 10
クラスの1つは、場で話すことであり、私たちの最初の課題の1つはハウツースピーチでした。
ことであり、私たちの最初の課題の1つはハウツースピーチでした。
課題の1つはハウツースピーチでした。
ハウツースピーチでした。
jimaku: 10 to 17
    section start: 10, end: 12, text: 私が学校で受けている公の
クラスの1つは、場で話す
    section start: 12, end: 14, text: ことであり、私たちの最初の
課題の1つは
    ketsu: 17
    section start: 14, end: 17, text: ハウツースピーチでした。
17
詰め込もうとしました
jimaku: 17 to 23
    ketsu: 23
    section start: 17, end: 23, text: 私は5分でできることに脳を
詰め込もうとしました
23
jimaku: 23 to 25
    ketsu: 25
    section start: 23, end: 25, text: スノーボード
25
ついて多くのことを学びました5分でスピーチをしてから
学びましたスピーチをしてから
してから
jimaku: 25 to 29
    secti

100%|██████████| 2297/2297 [00:10<00:00, 215.32it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0_subtitle2.mp4



100%|█████████▉| 2496/2498 [02:45<00:00, 16.64it/s]WARNING:py.warnings:/usr/local/lib/python3.8/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0.mp4, 2764800 bytes wanted but 0 bytes read,at frame 2497/2498, at time 104.15/104.16 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

100%|██████████| 2498/2498 [02:46<00:00, 15.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0_subtitle2.mp4 

